In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,5"
import numpy as np
import keras
from keras.models import model_from_json
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
from keras import backend as K

list_of_models = ['wide_spi1']

def load_model(model_prefix):
    model_json_string = open(model_prefix+'.json').read()
    model = model_from_json(model_json_string)
    model.load_weights(model_prefix+'.h5')
    return model

In [3]:
model = load_model(list_of_models[0])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [4]:
def compute_equivalent_weights(W1, W2, new_W):
    l1, c1, f1 = W1.shape
    l2, c2, f2 = W2.shape
    new_l, new_c, new_f = new_W.shape
    for k in xrange(0, new_l):
        for c in xrange(0, new_c):
            for f in xrange(0, new_f):
                w = 0
                lb = max(0, k-(l1-1))
                ub = min(k+1, l2)
                for j in xrange(lb, ub):
                    for cc in xrange(0, f1):
                        w = w + (W1[k-j][c][cc]*W2[j][cc][f])
                new_W[k][c][f] = w
    return new_W

In [5]:
# small test for compute_equivalent_weights

W1 = np.array([1,2,3,4]).reshape((2,1,2))
W2 = np.array([5,6]).reshape((1,2,1))
new_W = np.zeros((2,1,1))
new_W = compute_equivalent_weights(W1, W2, new_W)
print(W1)
print(W2)
print(new_W)

[[[1 2]]

 [[3 4]]]
[[[5]
  [6]]]
[[[17.]]

 [[39.]]]


In [6]:
from keras.models import Sequential
from keras.layers import Conv1D

def reparameterize (model):

    # figure out old conv layers
    config1 = model.layers[0].get_config()
    config2 = model.layers[1].get_config()
    W1 = model.layers[0].get_weights()[0]
    W2 = model.layers[1].get_weights()[0]

    filters = config2['filters']
    kernel_size = config1['kernel_size'][0] + config2['kernel_size'][0] - 1
    padding = config2['padding']
    activation = config2['activation']
    strides = config2['strides']
    input_shape = config1['batch_input_shape'][1:]
    
    # set up new conv layer
    new_model = Sequential()
    new_model.add(Conv1D(input_shape=input_shape,
                         filters=filters,
                         kernel_size=kernel_size,
                         padding=padding,
                         activation=activation,
                         strides=strides))
    
    # get weights for new conv layer
    new_l, new_c, new_f = new_model.layers[0].get_weights()[0].shape
    new_W = np.zeros((new_l, new_c, new_f))
    new_W = compute_equivalent_weights(W1, W2, new_W)
    
    # get bias for new conv layer
    new_b = model.layers[1].get_weights()[1]
    
    # set weight and bias
    new_model.layers[0].set_weights((new_W, new_b))
    
    # copy over rest of model
    for layer in model.layers[2:]:
        new_layer = layer.__class__.from_config(layer.get_config())
        new_layer.build(layer.input_shape)
        print(new_layer.weights)
        print(layer.weights)
        new_layer.set_weights(layer.get_weights())
        new_model.add(new_layer)
        

    return new_model

In [7]:
new_model = reparameterize (model)

[]
[]
[]
[]
[<tf.Variable 'separable_fc_3_W_pos:0' shape=(10, 25) dtype=float32_ref>, <tf.Variable 'separable_fc_3_W_chan:0' shape=(10, 1) dtype=float32_ref>]
[<tf.Variable 'separable_fc_3/separable_fc_3_W_pos:0' shape=(10, 25) dtype=float32_ref>, <tf.Variable 'separable_fc_3/separable_fc_3_W_chan:0' shape=(10, 1) dtype=float32_ref>]
[<tf.Variable 'kernel:0' shape=(10, 1) dtype=float32_ref>, <tf.Variable 'bias:0' shape=(1,) dtype=float32_ref>]
[<tf.Variable 'dense_3/kernel:0' shape=(10, 1) dtype=float32_ref>, <tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32_ref>]
[]
[]


In [21]:
from keras.models import Model
m1_conv = Model(inputs=model.layers[0].input,
               outputs=model.layers[2].output)


m2_conv = Model(inputs=new_model.layers[0].input,
                outputs=new_model.layers[1].output)

differences = np.abs(m1_conv.predict(test_set[:50])-m2_conv.predict(test_set[:50])) > 0.00001
nonzeros = np.nonzero(differences)

In [23]:
model.layers[0].get_config()

{'activation': 'linear',
 'activity_regularizer': None,
 'batch_input_shape': (None, 1000, 4),
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'dilation_rate': (1,),
 'dtype': u'float32',
 'filters': 32,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': u'uniform',
   'mode': u'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'kernel_size': (16,),
 'name': u'conv1d_5',
 'padding': u'same',
 'strides': (1,),
 'trainable': True,
 'use_bias': False}

In [22]:
zip(*nonzeros)

[(1, 3, 0),
 (2, 993, 0),
 (2, 994, 0),
 (4, 995, 0),
 (4, 998, 0),
 (5, 1, 0),
 (5, 997, 0),
 (10, 994, 0),
 (11, 995, 0),
 (13, 994, 0),
 (14, 0, 0),
 (14, 5, 0),
 (15, 0, 0),
 (15, 5, 0),
 (15, 994, 0),
 (16, 993, 0),
 (19, 0, 0),
 (19, 994, 0),
 (23, 0, 0),
 (24, 5, 0),
 (24, 994, 0),
 (25, 998, 0),
 (27, 0, 0),
 (28, 994, 0),
 (29, 1, 0),
 (29, 997, 0),
 (30, 995, 0),
 (32, 995, 0),
 (35, 996, 0),
 (37, 0, 0),
 (37, 3, 0),
 (38, 0, 0),
 (39, 0, 0),
 (39, 993, 0),
 (40, 2, 0),
 (41, 0, 0),
 (41, 996, 0),
 (42, 5, 0),
 (42, 994, 0),
 (44, 994, 0),
 (46, 1, 0),
 (47, 996, 0)]

In [ ]:
model.predict(test_set[:10])

In [ ]:
new_model.predict(test_set[:10])

In [ ]:
randin = np.random.random((10,1000,4))
model.predict(randin)

In [ ]:
np.array(model.layers[4].get_weights()[1]) - np.array(new_model.layers[3].get_weights()[1])

In [ ]:
model.layers[4].get_weights()

In [9]:
# small test for reparameterize

from momma_dragonn.data_loaders.hdf5_data_loader import MultimodalAtOnceDataLoader
data = MultimodalAtOnceDataLoader('test_data.hdf5')
test_set = np.array(data.X['sequence'][50:100])
test_labels = np.array(data.Y['output'][50:100])

model_preds = model.predict(test_set)
new_model_preds = new_model.predict(test_set)

tol = 0.00001
fail = False
for i, e in np.ndenumerate(model_preds):
    if abs(e - new_model_preds[i]) > tol:
        fail = True
        print("FAILED: case ", i[0], e, new_model_preds[i])
if(fail == False):
    print("SUCCESS!")

Input modes [u'sequence']
Output modes [u'output']
FAILED: case  1 0.39401627 0.39200103
FAILED: case  2 0.15938604 0.15784913
FAILED: case  5 0.18128243 0.16244912
FAILED: case  10 0.16516298 0.15933818
FAILED: case  11 0.28714067 0.28160957
FAILED: case  14 0.75377023 0.7325093
FAILED: case  15 0.82159644 0.8211974
FAILED: case  16 0.2690119 0.2648076
FAILED: case  19 0.9966799 0.9965077
FAILED: case  23 0.76464844 0.75866586
FAILED: case  24 0.2794227 0.2731783
FAILED: case  25 0.4613918 0.4547862
FAILED: case  27 0.99787724 0.9977863
FAILED: case  28 0.19252354 0.1857999
FAILED: case  29 0.25868192 0.25697985
FAILED: case  30 0.13521314 0.12945192
FAILED: case  32 0.31833413 0.30927566
FAILED: case  35 0.355323 0.33726615
FAILED: case  37 0.085916184 0.0820961
FAILED: case  38 0.5797198 0.5779358
FAILED: case  39 0.22677837 0.19126107
FAILED: case  40 0.5823371 0.57168263
FAILED: case  41 0.41910475 0.4080838
FAILED: case  42 0.17202832 0.16665243
FAILED: case  44 0.114011705 0.112